<a href="https://colab.research.google.com/github/mrahimi14/Text_Analysis/blob/main/Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hazm import Normalizer, Lemmatizer, word_tokenize, stopwords_list
from collections import Counter
from sklearn.cluster import KMeans
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Defining the functions:
def remove_emojis(text):
    # Emoji regex pattern to capture all emojis, including sequences
    emoji_pattern = re.compile(
        "[\U00010000-\U0010ffff"
        "|\U00002702-\U000027B0"    # Additional ranges for various symbols
        "|\U000024C2-\U0001F251"    # Enclosed characters like circled letters and numbers
        "|\U0001F004-\U0001F0CF"    # Playing card symbols
        "|\U0001F200-\U0001F251"    # Other symbols
        "|\U0000203C|\U0000FE0F]",  # Special characters and variation selectors
        flags=re.UNICODE)

    # Replace emojis with empty string
    return emoji_pattern.sub(r'', text)

In [ ]:
# Sample Persian fake news data
fake_news_persian = [
    '⛅️🌤💨 تلخ از تو شیرین میشود کفر از تو چون دین میشود خار از تو نسرین میشود چیزی بده درویش را جانِ من و جانان من کفر من و ایمان من سلطانِ سلطانان من چیزی بده درویش را #مولانا📖 با آرزوی آدینه ای سرشار ازآرامش؛ از شما دعوت می شود که تاریخ امروز ایران و جهان را مرور نمائید👇👇 #خورشیدی: آدینه، ۱۴ دی ۱۴۰۳/۱۰/۱۴ #میلادی: Friday, January 3, 2025-01-03 قمری: الجمعة، ٢ رجب ١٤٤٦/۰۷/۰۲ #روزها🌈 💃روز جشن سیرسور جشن گياه‌خواری ايرانيان #رویدادها👁‍🗨 🔴۱۲۸۶ خ- روزنامه صور اسرافیل با همکاری میرزا جهانگیرخان شیرازی و قاسم‌خان تبریزی با همکاری میرزا علی اکبر خان قزوینی (دهخدا) صور اسرافیل را منتشر کردند🗞 🔴۱۲۹۲ خ- پرواز اولین هواپیما در آسمان ایران توسط کوزمینسکی، خلبان لهستانی‌الاصل   در پی اجرای يك برنامه پرواز نمايشی در آسمان تهران توسط خلبانی لهستانی با يك هواپيمای دوباله سبك در زمان احمدشاه🛩 🔴۱۳۰۰ خ- تشکیل ارتش شاهنشاهی ایران⚔🛡 🔴۱۳۵۷ خ-  آغاز کنفرانس گوادلوپ دربارهٔ انقلاب ۱۳۵۷ ایران🧐 🔵۱۵۲۱ م- پاپ لئوی دهم، مارتین لوتر را تکفیر و از کلیسای کاتولیک رم طرد کرد⛪️ 🔵۱۹۹۰ م- ژنرال مانوئل نوریگا پس از مداخله نظامی آمریکا از رهبری پاناما خلع شد🚶‍♂️ #زادروزها💚 🟢۱۱۸۶ خ- محمدشاه قاجار، سومین شاه ایران از قاجاریان👑 🟢۱۳۱۸ خ- حسین الهی قمشه‌ای ، نویسنده، مترجم و سخنران ایرانی، مدرس دانشگاه و فرزند مهدی الهی قمشه‌ای✍ 🟢۱۳۲۰ خ- همایون بهزادی، فوتبالیست سابق تیم ملی ایران، شاهین و پرسپولیس معروف به سر طلایی✨ 🟢۱۳۲۳ خ- سیما بینا، نوازنده، نقاش، آهنگساز و خوانندهٔ آوازها و ترانه‌های محلی ایرانی💃 🟢۱۳۲۵ خ-ابوالحسن خوشرو خواننده و نوازنده موسیقی مازندرانی🎙 🟢۱۳۳۷ خ- فاطمه صولتی با نام هنری شهره، خواننده و بازیگر ایرانی او همچنین خواهر شهرام صولتی است💃 🟢۱۸۹۲ م- جان رونالد روئل تالکین، نویسنده و زبان‌شناس بریتانیایی، خالق آثاری چون هابیت و ارباب حلقه‌ها📚 🟢۱۹۵۶ م- مل گیبسون بازیگر، کارگردان و تهیه‌کننده استرالیایی-آمریکایی، برندهٔ جایزه اسکار🎖 🟢۱۹۶۹ م- مایکل شوماخر رانندهٔ آلمانی مسابقات فرمول یک🏎 #درگذشت‌ها🖤 🟤۱۳۸۴ خ- ترور بیولوژیک سعید کاظمی آشتیانی، دانشمند ایرانی، بنیانگذار و رئیس سابق پژوهشگاه رویان🦠 🟤۱۳۸۹ خ- علی‌رضا پهلوی، فرزند محمدرضا پهلوی🚶‍♂️ 🟤۱۹۴۵ م- ادگار کیسی، پیشگو و شفادهندهٔ آمریکایی، وی را «پیامبر خفته» و «بزرگ‌ترین عارف آمریکایی» نیز خوانده‌اند. 🤷‍♂ 🟤۱۹۵۶ م- یوزف ویرت سیاستمدار آلمانی، جوانترین صدراعظم این کشور از سال ۱۹۲۱ تا ۱۹۲۲🧑‍🦱 🟤۱۹۶۷ م- جک رابی عامل ترور لی هاروی (عامل ترورجان اف. کندی رئیس‌جمهور آمریکا)😒 🟤۲۰۰۹ م- پت هینگل بازیگر آمریکایی🎭 ☘ تنظیم و تحقیق: حسین علی‌اکبری🕊 ویرایش و تدوین: #فردین✍ @IR_YARAN⚓️ 🌿🍂🌿🍂🌿🍂🌿🍂🌿🍂🌿🍂',
    '⛅️🌤💨 تلخ از تو شیرین میشود کفر از تو چون دین میشود خار از تو نسرین میشود چیزی بده درویش را جانِ من و جانان من کفر من و ایمان من سلطانِ سلطانان من چیزی بده درویش را #مولانا📖 با آرزوی آدینه ای سرشار ازآرامش؛ از شما دعوت می شود که تاریخ امروز ایران و جهان را مرور نمائید👇👇 #خورشیدی: آدینه، ۱۴ دی ۱۴۰۳/۱۰/۱۴ #میلادی: Friday, January 3, 2025-01-03 قمری: الجمعة، ٢ رجب ١٤٤٦/۰۷/۰۲ #روزها🌈 💃روز جشن سیرسور جشن گياه‌خواری ايرانيان #رویدادها👁‍🗨 🔴۱۲۸۶ خ- روزنامه صور اسرافیل با همکاری میرزا جهانگیرخان شیرازی و قاسم‌خان تبریزی با همکاری میرزا علی اکبر خان قزوینی (دهخدا) صور اسرافیل را منتشر کردند🗞 🔴۱۲۹۲ خ- پرواز اولین هواپیما در آسمان ایران توسط کوزمینسکی، خلبان لهستانی‌الاصل   در پی اجرای يك برنامه پرواز نمايشی در آسمان تهران توسط خلبانی لهستانی با يك هواپيمای دوباله سبك در زمان احمدشاه🛩 🔴۱۳۰۰ خ- تشکیل ارتش شاهنشاهی ایران⚔🛡 🔴۱۳۵۷ خ-  آغاز کنفرانس گوادلوپ دربارهٔ انقلاب ۱۳۵۷ ایران🧐 🔵۱۵۲۱ م- پاپ لئوی دهم، مارتین لوتر را تکفیر و از کلیسای کاتولیک رم طرد کرد⛪️ 🔵۱۹۹۰ م- ژنرال مانوئل نوریگا پس از مداخله نظامی آمریکا از رهبری پاناما خلع شد🚶‍♂️ #زادروزها💚 🟢۱۱۸۶ خ- محمدشاه قاجار، سومین شاه ایران از قاجاریان👑 🟢۱۳۱۸ خ- حسین الهی قمشه‌ای ، نویسنده، مترجم و سخنران ایرانی، مدرس دانشگاه و فرزند مهدی الهی قمشه‌ای✍ 🟢۱۳۲۰ خ- همایون بهزادی، فوتبالیست سابق تیم ملی ایران، شاهین و پرسپولیس معروف به سر طلایی✨ 🟢۱۳۲۳ خ- سیما بینا، نوازنده، نقاش، آهنگساز و خوانندهٔ آوازها و ترانه‌های محلی ایرانی💃 🟢۱۳۲۵ خ-ابوالحسن خوشرو خواننده و نوازنده موسیقی مازندرانی🎙 🟢۱۳۳۷ خ- فاطمه صولتی با نام هنری شهره، خواننده و بازیگر ایرانی او همچنین خواهر شهرام صولتی است💃 🟢۱۸۹۲ م- جان رونالد روئل تالکین، نویسنده و زبان‌شناس بریتانیایی، خالق آثاری چون هابیت و ارباب حلقه‌ها📚 🟢۱۹۵۶ م- مل گیبسون بازیگر، کارگردان و تهیه‌کننده استرالیایی-آمریکایی، برندهٔ جایزه اسکار🎖 🟢۱۹۶۹ م- مایکل شوماخر رانندهٔ آلمانی مسابقات فرمول یک🏎 #درگذشت‌ها🖤 🟤۱۳۸۴ خ- ترور بیولوژیک سعید کاظمی آشتیانی، دانشمند ایرانی، بنیانگذار و رئیس سابق پژوهشگاه رویان🦠 🟤۱۳۸۹ خ- علی‌رضا پهلوی، فرزند محمدرضا پهلوی🚶‍♂️ 🟤۱۹۴۵ م- ادگار کیسی، پیشگو و شفادهندهٔ آمریکایی، وی را «پیامبر خفته» و «بزرگ‌ترین عارف آمریکایی» نیز خوانده‌اند. 🤷‍♂ 🟤۱۹۵۶ م- یوزف ویرت سیاستمدار آلمانی، جوانترین صدراعظم این کشور از سال ۱۹۲۱ تا ۱۹۲۲🧑‍🦱 🟤۱۹۶۷ م- جک رابی عامل ترور لی هاروی (عامل ترورجان اف. کندی رئیس‌جمهور آمریکا)😒 🟤۲۰۰۹ م- پت هینگل بازیگر آمریکایی🎭 ☘ تنظیم و تحقیق: حسین علی‌اکبری🕊 ویرایش و تدوین: #فردین✍ @IR_YARAN⚓️ 🌿🍂🌿🍂🌿🍂🌿🍂🌿🍂🌿🍂 ',
    'گروه مدلل (مالک برند غنچه) شرکت صافولا بهشهر(مالک برند لادن) را به ارزش ۲۰۰ میلیون دلار خریداری کرد 🔸 صافولا یک هولدینگ بین‌المللی تولید مواد غذایی عربستان سعودی است که دارای زیر مجموعه‌هایی در کشورهای خاورمیانه و شمال آفریقا می‌باشد. ۱۰ درصد از سهام این شرکت، در اختیار دولت عربستان سعودی می‌باشد. #پایگاه_خبری_بامداد @Bmneews1',
    'توضیحات کامل شهردار رباط‌کریم در خصوص ۲ پروژه شاخص در آستانه افتتاح / رمپ و لوپ پل الارد؛ رفع گره‌‌ای مهم در راستای تسهیل تردد شبکه ترافیکی 👤 میرجعفریان معاون هماهنگی امور عمرانی استانداری تهران با حضور در شهرستان رباط‌کریم و با همراهی مسئولان عالی شهرستانی، دکتر الهیاری شهردار رباط‌کریم از پروژه‌های رمپ و لوپ پل الارد در آزادراه تهران _ ساوه و پایانه حمل و نقل اتوبوسرانی شهرداری رباط‌کریم بازدید کرد. 🚩 دکتر الهیاری شهردار رباط‌کریم در جریان این بازدید و نظارت میدانی، توضیحات کاملی را در خصوص جزئیات و فرآیند پیشرفت این پروژه‌ها ارائه کرد و اجرای دقیق مطابق موازین فنی و مهندسی و پیش‌بینی تمامی جوانب و امکانات لازم در راستای بهره‌برداری کامل را از شاخصه‌های مهم این پروژه‌ها برشمرد. پایگاه_خبری_بهارستان_نیوز 🗓 ۱۵ دی ماه ۱۴۰۳ 🔹 بهارستان نیوز 📍با کانال #بهارستان_نیوز همراه باشید ... ⓙⓞⓘⓝ↯ @Baharestan_Newss',
    'آقاى [جلیل] بشارتى، [رییس دفتر مناطق محروم ریاست جمهوری] آمد. گزارش سفر به منطقه‌اى مرزى و محروم اطراف خواف و عمران روستاها و امتنان روستایيان شيعه و سنى را داد. ✍آیت الله هاشمی در آینه خاطرات 🗓شنبه ۱۵ دى ۱۳۷۵ 💠ساعت هفت و نیم صبح به دفترم رفتم. خيابان‌ها به خاطر تعطيلات زمستاني مدارس، خلوت است. تا ساعت نُه و نیم،كارهاى عقب مانده را انجام دادم. آشيخ محمد آخوندى، [مؤسس دارالکتب‏ الاسلامیه] آمد. از خدمات زيربنايى دولت، اظهار اعجاب كرد و براى ساخت دو هزار خانه ارزان‏قيمت و مدرسه علميه استمداد نمود. 💠[آقای سیدمحمدکاظم حسنی‏ طباطبایی]، سفير جديدمان در كنيا براى خداحافظى و كسب نظر آمد. آقاى [جلیل] بشارتى، [رییس دفتر مناطق محروم ریاست جمهوری] آمد. گزارش سفر به منطقه‌اى مرزى و محروم اطراف خواف و عمران روستاها و امتنان روستایيان شيعه و سنى را داد. آقاى [سیدابوالفضل] موسوى‏تبريزى آمد. از وضع دولت بعدي، اظهارنگرانى كرد و مذاكراتى در مورد صادرات فرش و كارخانه شيشه آذر در تبريز داشت. عصر شوراى عالى جوانان، جلسه با چند مصوبه داشت. شب كارهاى مانده از سفر را انجام دادم و در دفترم خوابيدم. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    '‍ موسوی‌ لاری، وزير كشور و معاونش تاج‌زاده، گزارشی از پيشرفت كارهای انتخابات دادند و از احتمال حذف وسيع در شورای نگهبان، اظهار نگرانی كردند. ✍آیت الله هاشمی در آینه خاطرات 🗓چهار‌شنبه ۱۵ دی ۱۳۷۸ 💠ديشب و امروز مقداري باران داشتيم و هواي آلوده تميز شد. از داخل اتاقم، بعد از هفته‌ها، چراغ‌هاي شهر شفاف و درخشان ديده مي‌شود. مطابق معمول هر شب، ساعت ده و نیم خوابيدم و ساعت سه و نیم بامداد، براي سحر بيدار شدم. سماور و زودپز را براي گرم‌شدن پلو روشن كردم. تا ساعت پنج، به دعاهاي سحر و قرائت قرآن گذشت. كاظم [مرعشی، برادر همسرم که با ما زندگی می‌کند] را بيدار كردم. ميز را در آشپزخانه آماده كرد. با هم سحري خورديم و بعد از مسواك و تجديد وضو و نماز، ساعت شش صبح خوابيدم. خلاصه خبر راديو بي بي سي را گرفتم؛ موذيانه و به نفع جريان‌هاي ضداسلام در ايران عمل مي‌كنند. 💠ساعت هشت و نیم صبح به دفترم رسيدم. روزنامه‌ها  و گزارش‌ها را از روي كامپيوتر ديدم؛ تقريباً اكثر روزها برنامه همين است. ساعت نه و نیم، دكتر [علی‌اکبر] ولايتي، [مشاور رهبری در امور بین‌الملل] آمد. گفت، گروه‌هاي همسو با [جامعه] روحانيت [مبارز]، براي هماهنگ‌كردن برنامه‌هاي انتخابات، شوراي عالي تشكيل داده‌اند و پيغام آورد كه به [حزب] كارگزاران [سازندگی]، توصيه كنم كه با چپ‌ها ائتلاف نكنند و چند نفر از جناح روحانيت را هم در ليست داشته باشند. خواست، در مراسم فارغ‌التحصيلي دانشگاه شهيد بهشتي شركت كنم. گفتم، آن روزها در قم هستم و ممكن است، از لحاظ قانون انتخابات هم مشكل‌زا شود؛ قرار شد بررسي شود. 💠آقاي [محمدرضا] عليزاده، ریيس‌كل [سازمان] ثبت اسناد [و املاک کشور] آمد. مطابق معمول مديران جديد، نمونه‌هايي از ضعف‌هاي ریيس قبل -آقاي [سیدرضا] زواره‌اي- و اصلاحات خود را داد؛ البته توضيحات مفيدي داشت. 💠[آقای منصور معظمی]، سفير جديدمان در برزيل آمد.از مديران صنايع پتروشيمي است. ابتدا گزارشي از پيشرفت اين صنعت در دوران سازندگي داد و اظهار تأسف كرد كه بعد [از دولت سازندگی]،كار جديدي نشده است، ولي خوشبختانه آقاي [محمدرضا] نعمت‌زاده، [مدیرعامل شرکت ملی صنایع پتروشیمی]، طرح‌هاي جديدي در دست توافق دارد كه مي‌تواند در سال‌هاي آينده، ثمراتش ظاهر شود. سپس از تجربيات سازندگي برزيل گفت که ۱۶٠ ميليارد دلار بدهي بار آورده است. 💠عصر آقاي [غلامحسین] رهبرپور، [رییس دادگاه‌های انقلاب استان تهران] آمد. از مشكلات دادگاه انقلاب و بي‌توجهي‌ها نسبت به آن گفت. [آقای عبدالواحد موسوی‌لاری]، وزيركشور و معاونش آقاي [مصطفی] تاج‌زاده، [رییس ستاد انتخابات]، آمدند. گزارشي از پيشرفت كارهاي انتخابات دادند و از احتمال حذف وسيع در شوراي نگهبان، اظهار نگراني كردند. جناح چپ، تلاش وسيعي را براي مهار شوراي نگهبان شروع كرده است. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    '‍ اظهار امیدواری عرفات در مورد توافق با اسراییل/هشدارسران ١۴ گروه فلسطینی به عرفات/مطالعه پیش نویس لایحه فقر زدایی، همان لایحه سال ٧۵ است دولت خاتمی آنرا مسترد کرد، ۴ سال وقت ضایع شد ✍آیت الله هاشمی در آینه خاطرات 🗓پنج‌شنبه ١۵ دی ١٣٧٩ 💠در كوشك بودم. بيشتر وقت به مطالعه گذشت. نم نم باران هم داشتيم. در گزارش‌ها بود كه [یاسر] عرفات، بعد از ملاقات با [حُسنی] مبارك، [رییس‌جمهور مصر]، گفته است،احتمال رسيدن به توافق با پشتكار كلينتون با اسرایيل در باقيمانده دوره رياست‌جمهوري او وجود دارد،ولي سران مجموعه ١۴ گروه‌ مبارز فلسطيني، به او هشدار داده‌اند كه از دادن امتياز به اسرایيل، پرهيز كند. 💠پيش‌نويس لايحه فقرزدايي دولت را خواندم؛ تقريباً به مضمون همان لايحه‌اي است كه دولت سازندگي [با محوریت اشتغال در سال ١٣٧۵] به مجلس فرستاد.دولت آقاي [سیدمحمد] خاتمي، در شروع كار،آن را مسترد داشت و چهارسال، وقت ضايع شد. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی',
    'در اخبار شب آمد كه آمريكا دو جنگنده ليبي را در مديترانه سرنگون كرده است ✍آیت الله هاشمی در آینه خاطرات 🗓چهارشنبه ۱۴ دی ۱۳۶۷ 💠مطابق معمول قبل از رفتن به جلسه علني گزارش‌ها را خواندم و صبحانه خوردم. در جلسه علني تا آخر وقت ماندم. طرح تسهيلات براي رزمندگان در دستور بود. در تنفس براي كاركنان مجتمع صنعتي گوشت فارس [و جمعی از اهالی مرودشت] كه پانصد هزار تومان براي جنگ آورده بودند و خانواده‌هاي رزمندگان لشكر 5 نصر صحبت كردم . عصر هيأت رئيسه مجلس جلسه داشت. 💠در اخبار شب آمد كه آمريكا دو جنگنده ليبي را در مديترانه سرنگون كرده است . شب جامعه روحانيت و مجمع روحانيون مبارز، در دفتر آقاي خامنه‌اي جلسه مشترك داشتند. من و آقايان خامنه‌اي، [سید محمد] موسوي خوئيني‌ها و [سید محمد] خاتمي صحبت كرديم. جلسه خوبي بود. قرار شد ادامه داشته باشد و خبري منتشر كرديم. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی',
    'محسن آمد. درباره سيمان و شكر صحبت كرد. مي‌خواست [آموزش] كامپيوتر بدهد كه وقت نبود./با دكتر ولايتي و مهندس [میرحسین] موسوي درباره مسائل مرزي با آذربايجان شوروي صحبت شد. ✍آیت الله هاشمی در آینه خاطرات 🗓پنجشنبه ۱۴ دی ۱۳۶۸ 💠بعد از نماز تا ساعت نه صبح كارهاي دفتر انجام شد. دكتر [علی] عباسپور آمد و گزارش وضع صنايع [پدافند] ش. م . ر [شیمیایی، میکروبی و رادیواکتیو] را داد. آقاي [محمد] اصغري [مدیر مسئول روزنامه کیهان] آمد. وضع [مؤسسه] كيهان را گفت و از پيشنهادهاي دفتر آيت‌الله خامنه‌اي در كيفيت نقل اخبار رهبري گفت. آقايان سعيد الذاكرين و رئيس زاده از روابط عمومي مجلس آمدند. پيشنهاد انتقال از مجلس به دستگاه اجرایي دارند. دكتر حبيبي براي كارهاي جاري كشور آمد. عصر كميسيون خاص براي تفكيك وظايف جهاد و كشاورزي آمدند. آنها طرح تصويب شده در كميسيون را توضيح دادند و مشورت كردند؛ آب و خاك و منابع طبيعي را به جهاد و امور كشاورزي و آموزش را به وزارت كشاورزي داده‌اند. گفتم با كميسيون دولت هماهنگي كنند. 💠محسن آمد. درباره سيمان و شكر صحبت كرد. مي‌خواست [آموزش] كامپيوتر بدهد كه وقت نبود. تا ساعت هشت شب در دفترم كار كردم. با دكتر ولايتي و مهندس [میرحسین] موسوي درباره مسائل مرزي با آذربايجان شوروي صحبت شد. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی',
    'دكتر ملكى و برادرش آمدند؛ هر دو پزشك هستنند و در كانادا كار مى‌كنند. گفتند در ايران از داروهاى مدرن استفاده نمى‌شود. از ضرورت استفاده از ابزار پيشرفته پزشكى و داروهاى مدرن و تخصص بالاى ايرانى‌هاى موجود در خارج گفتند ✍آیت الله هاشمی در آینه خاطرات 🗓شنبه ۱۴ دى ۱۳۷۰ 💠ديشب برف خوبى باريده است. كسالت عفت ادامه داشت. تنهايى صبحانه خوردم و براى عفت شير گرم كردم. 💠ساعت هشت و نيم صبح به دفترم رسيدم. آقاى [حسين]محلوجى، [وزير معادن و فلزات]، مديرعامل كارخانه ذوب فولاد اصفهان و مديرعامل فولاد مباركه آمدند. براى انعقاد قرارداد توسعه فولاد اصفهان با ايتاليا، اجازه خواستند؛ گويا طرحى كه ايتاليايي‌ها قرار بوده، قبل از تجاوز عراق به كويت، براى بصره اجرا كنند و اكنون فسخ شده، مى‌خواهند با بهاى كمتر به‌ ما بدهند. راجع به فولاد مباركه و هزينه‌هاى ارزى آن مذاكره شد؛ به‌نظرم خيلى زياد است و نياز به توضيح دارد. 💠دكتر ملكى و برادرش آمدند؛ هر دو پزشك هستنند و در كانادا كار مى‌كنند. گفتند در ايران از داروهاى مدرن استفاده نمى‌شود. از ضرورت استفاده از ابزار پيشرفته پزشكى و داروهاى مدرن و تخصص بالاى ايرانى‌هاى موجود در خارج گفتند. برادرش تقاضا دارد كه كمك كنيم، با بورس دولتى در دانشگاه كانادا ادامه تحصيل بدهد. 💠دكتر [حسن]حبيبى، [معاون اول رييس جمهور] براى امور جارى كشور و تأسيس واحد اطلاع رساني در نهاد آمد. دكتر [محمدحسين]عادلى، [رييس بانك‌مركزي] آمد. گزارشى از وضع ارز و ريال داد و برنامه كنترل نقدينگى را گفت. در مورد پشتيبانى از همكاري‌هاى اقتصادى و تجارى با جمهورى‌هاى مسلمان‌نشين شوروى سابق صحبت شد. 💠عصر تا ساعت شش كار كردم. شوراى عالى امنيت ملي جلسه داشت. دربارة مقابله به مثل در مقابل شيطنت‌هاى فرانسه و سوييس صحبت شد. بعد از جلسه با دكتر ولايتى، [وزير امورخارجه] و فلاحيان[وزير اطلاعات]، دربارة افغانستان مذاكره كرديم. ✍️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی',
    'عصر شوراى اقتصاد، به عنوان شوراى عالى مناطق آزاد جلسه داشت كه تصميماتى در مورد جلوگيرى قاچاق از جزيره قشم و مقررات ورود كالا از جزيره كيش به داخل داشتيم. ✍آیت الله هاشمی در آینه خاطرات 🗓دوشنبه ۱۴ دى ۱۳۷۱ 💠ديشب كمى برف باريده است. ساعت نُه صبح، آقايان [عبدالحليم] خدام، [معاون رييس‌جمهور] و فاروق‌الشرع، [وزير امور خارجه سوريه] آمدند. مذاكرات درباره فلسطين و عراق و خليج فارس بود؛ مطلب تازه‌اى نداشتند. 💠[آقايان احمد طاها، عبدالرحيم نوربخش، محمد عباس‌پور و رسول پورزمان]، نمايندگان بوكان، مهاباد، ماكو و نقده آمدند و براى حوزه‌هاى انتخابيه خود استمداد كردند. اعضاي كميسيون صنايع مجلس آمدند. نظراتى دربارة صنعت داشتند و خواستار حمايت بيشتر از صنعت در بودجه 1372 و برنامه پنج‌ساله دوم شدند. 💠ظهر آقايان [محمدعلي] شرعى، طاهرى، قاضى، [محمد] مؤمن و [مصطفي] محقق داماد از قم آمدند؛ براى رفع مشكلاتى كه در خصوص مدرسه عالى تربيت‌ قاضى و معلم با وزارت فرهنگ و آموزش عالى دارند، استمداد كردند و براي شهرك مهديه هم كمك خواستند. 💠عصر شوراى اقتصاد، به عنوان شوراى عالى مناطق آزاد جلسه داشت كه تصميماتى در مورد جلوگيرى قاچاق از جزيره قشم و مقررات ورود كالا از جزيره كيش به داخل داشتيم. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'به عيادت آيت‌الله [سيدعبدالكريم] موسوى اردبيلى در بيمارستان خاتم‌الانبياء رفتم. با سكته ناقص، طرف چپ بدنش بى‌حركت شده و با اشكال صحبت مى‌كند؛ رو به خوبى است. بچه‌هايش و دامادش آقاي قاسم‌زاده هم بودند/آقاى نيازف تحت تأثير اطلاعات و استعداد مهدى قرار گرفته بود. در مورد مسايل افغانستان پرسيد و مشورت كرد؛ گفتم ما سياست بى‌طرفى در جنگ‌هاى داخلى آنها را داريم. او گفت آنها هم بى‌طرف‌اند. گفت روسيه و ژنرال دوستم از او خواسته‌اند كه ترتيب ملاقات با سران نظامي و رؤساى روسيه در تركمنستان را بدهد. ✍آیت الله هاشمی در آینه خاطرات 🗓سه‌شنبه ۱۴ دى ۱۳۷۲ 💠ديشب و امروز هم باران داشتيم. ميهمانانمان امروز به مازندران رفتند و از گنبد و كلاله و رامسر بازديد كردند. تحت تأثير طبيعت غنى مازندران قرار گرفته̛بودند. مهدى هم همراهشان رفته بود. ساعت نه‌ونيم صبح، [آقای غلامرضا آقازاده]، وزير نفت آمد و براى افتتاح ال‌سي در واحد تصفيه گاز در سرخس، به اندازه شانزده ميليون مترمكعب در روز استمداد كرد و براى اجاره يك دكل حفارى در درياى خزر از جمهوري آذربايجان و همچنين واگذارى طرح سكوى ابوذر به شركت‌هاى داخلى، به خاطر طفره رفتن پيمانكار آمريكايى از دادن سي درصد گاز به داخل اجازه خواست؛ همه را پذيرفتم. در مورد بهره‌گيرى از پارس جنوبى هم ايده جديدى را مطرح كرد. 💠 آقاى [مصطفي] ميرسليم، [دبیر شورای عالی انقلاب فرهنگی]، براى تنظيم دستور شوراى عالى انقلاب فرهنگى آمد. آقاى گودرزى آمد و براى پرسنل تخريب حفاظت كمك خواست. هيأت امناى دانشگاه شاهد جلسه داشت. طرح ساختمان مركزى را تصويب كرديم و تعدادى ديگر مصوبات داشتيم و گزارش كار شنيدم. 💠فاطى آمد. مطابق معمول چند نامه داشت. سرشب [آقاي اسحاق جهانگيري]، استاندار اصفهان آمد و براى كارهاى استان استمداد كرد. در مورد علت منظورنشدن در هيأت مديره منطقه آزاد قشم توضيح دادم. گفت آقاي نيازُف در ديدار از فولاد مباركه، تحت تأثير عظمت كارخانه قرار گرفت، ولى وقت زيادى براى بازديد از قسمت‌هاى مختلف نداشت. 💠شب به عيادت آيت‌الله [سيدعبدالكريم] موسوى اردبيلى در بيمارستان خاتم‌الانبياء رفتم. با سكته ناقص، طرف چپ بدنش بى‌حركت شده و با اشكال صحبت مى‌كند؛ رو به خوبى است. بچه‌هايش و دامادش آقاي قاسم‌زاده هم بودند. از آنجا به اقامتگاه آقاى نيازف رفتم كه از مازندران برگشته است؛ از تسهيلات سفرها و ملاقات با تركمن‌ها در گنبدكاووس، خيلى اظهار امتنان كرد. به شدت تحت تأثير مازندران و رامسر قرار گرفته بود. گفت چون خودشان از خدا دور شده‌اند، سواحل‌شان از اين نعمت‌ها ندارد. گفتم آقاى يلتسين را بياورد تا عظمت اينجا را ببينند. تحت تأثير اطلاعات و استعداد مهدى قرار گرفته بود. در مورد مسايل افغانستان پرسيد و مشورت كرد؛ گفتم ما سياست بى‌طرفى در جنگ‌هاى داخلى آنها را داريم. او گفت آنها هم بى‌طرف‌اند. گفت روسيه و ژنرال دوستم از او خواسته‌اند كه ترتيب ملاقات با سران نظامي و رؤساى روسيه در تركمنستان را بدهد. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'آقاى [غلامرضا] آقازاده، [وزیر نفت] آمد. نتيجه بررسى مسير خط لوله گاز ايران به پاكستان را آورد و گفت هزينه استحصال و انتقال براى هر يك ميليون B.T.U دو دلار است. هر متر مكعب حدود هفت سِنت و ارزان‌ترين گاز قابل وصول به پاكستان خواهد بود كه قيمت ارزش ذاتى گاز هم بايد اضافه شود. در مورد كيفيت تحقيقات و پيگيرى‌هاى وزارت اطلاعات، در مورد متهمان به فساد، اظهار ترديد كرد كه ممكن است اقدامى سياسى بوده و سوءنيت در كار باشد. گفت در هرمزگان، كنار اسكله آهن، يك پهلوگير براى نفت بايد احداث شود. ✍آیت الله هاشمی در آینه خاطرات 🗓چهارشنبه ۱۴ دى ۱۳۷۳ 💠 مثل اينكه روس‌ها مجبور به عقب نشينى از شهر گروزنى شده‌اند و با بمباران و توپخانه، شهر را مى‌زنند؛ مهم است. [آقای واختانگ گوگوادزه]، معاون پارلمان گرجستان آمد. اعضاى هیأت همراهش، دانشمند و خاورشناس و فرهنگى بودند. اكثراً فارسى مى‌دانند و مدعى علاقه به ايران هستند. بر لزوم تجديد روابط قديمى تأكيد كردم. گفت، صحبت‌هاى آقاى شواردنادزه، [رییس‏جمهور گرجستان]، بد نقل شده كه برداشت شد، عمليات نظامى روسيه عليه چچن را تأييد مى‌كند. گفت مخالف حمله نظامى هستيم، ولى بايد چچن‌ها هم قانون اساسى فدراسيون روسيه را مراعات كنند. اصرار به سفرم به گرجستان داشت. 💠دكتر [محمدعلی] نجفى، [وزیر آموزش و پرورش] آمد. نيازهاى آموزش و پرورش را گفت و براى كمبود بودجه استمداد كرد. در مورد كيفيت اداره دوره پيش‏ دانشگاهى در نظام جديد سئوال كرد که دولت اداره كند، يا به صورت غيرانتفاعى باشد؛ گفتم بيشتر بررسى شود. در مورد حق‏التدریسى‌ها هم نظر دادم كه بايد به تدريج جذب شوند. تذكر دادم كه اظهاراتش در مورد سازمان امور اداری و استخدامى را اصلاح كند كه آقاى ميرمحمدى، [دبیرکل سازمان امور اداری و استخدامی کشور] را عصبانى كرده است. آقاى [غلامرضا] آقازاده، [وزیر نفت] آمد. نتيجه بررسى مسير خط لوله گاز ايران به پاكستان را آورد و گفت هزينه استحصال و انتقال براى هر يك ميليون B.T.U دو دلار است. هر متر مكعب حدود هفت سِنت و ارزان‌ترين گاز قابل وصول به پاكستان خواهد بود كه قيمت ارزش ذاتى گاز هم بايد اضافه شود. در مورد كيفيت تحقيقات و پيگيرى‌هاى وزارت اطلاعات، در مورد متهمان به فساد، اظهار ترديد كرد كه ممكن است اقدامى سياسى بوده و سوءنيت در كار باشد. گفت در هرمزگان، كنار اسكله آهن، يك پهلوگير براى نفت بايد احداث شود. 💠مديران جهاد دانشگاهى آمدند. آقاى [محمد] رحمتى، [رییس جهاد دانشگاهی]، گزارش داد و من صحبت مفصلى كردم ؛ تأييدشان كردم و گفتم اجبار ندارند كه از دانشگاه خارج شوند، ولى اگر خواستند خارج شوند كمك‌شان مى‌كنم. توصيه به تعادل و پرهيز از افراط و تفريط كردم. سلمانى براى اصلاح آمد و از استخر هم استفاده كردم. تلفنى با عفت صحبت كردم. از مدينه به مكه آمده است. تا ساعت نُه شب كار كردم. به جلسه هیأت دولت نرفتم. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    '🔴 تلاش صلیبی هابرای نبش قبر پیامبر ﷺ و دزدیدن جسد ایشان. در سال 557 هجری شبی سلطان #نورالدین_زنگی رحمه الله در خوابش، پیامبر ﷺ را دید که به دو مرد سفید پوست مو بور اشاره می نمود و می فرمود: به من کمک کن، مرا از دست این دو نفر نجات بده! به همین خاطر با دلهره بیدار شد، سپس وضو گرفت، نماز خواند و خوابید. اما دوباره همان #خواب را دید، بنابراین دوباره بیدار شد، وضو گرفت، نماز خواند و خوابید. سپس برای بار سوم نیز همان خواب را دید، پس بیدار شد و گفت: دیگر وقت خواب نیست. او وزیر صالح و نیکوکاری داشت که نامش جمال الدین موصلی بود، پس به دنبال او فرستاد و ماجرای خواب را برایش بازگو کرد. او به سلطان گفت: چرا نشسته ای؟ هم اکنون بسوی مدینه منوره برو و آنچه دیده ای را پنهان دار. پس در آن وقت شب آماده شد و همراه وزیرش جمال الدین بسوی مدینه به راه افتاد. در حالی که اهل مدینه در مسجد نبوی جمع شده بودند، وزیر گفت: سلطان قصد زیارت پیامبر ﷺ را نموده و اموالی را برای دادن صدقه با خود آورده است. پس نام کسانی که مستحق #صدقه هستند را بنویسید. بنابراین اهل مدینه نام همگی آنها را نوشتند و سلطان دستور داد تا همه حضور یابند. او به تمام حاضران به دقت می نگریست تا صفتی را که پیامبر ﷺ به او نشان داده بود، با مردم تطبیق دهد، اما آن صفت را در آنان نیافت. پس از آن صدقات را توزیع نمود تا اینکه مردم بهره مند شدند و از آنجا رفتند. سلطان گفت: آیا کسی مانده که صدقه ای دریافت نکرده باشد؟ گفتند: نه. گفت: خوب فکر کنید. گفتند: فقط دو مرد مغربی مانده اند که چیزی از کسی دریافت نمی کنند و آن دو نیکوکار و ثروتمند هستند و بسیار به نیازمندان صدقه می دهند. پس دلش آرام گرفت و گفت: آن دو را نزدم بیاورید. آن دو نفر را نزدش آوردند. وقتی به آنها نگریست، دید همان دو مردی هستند که پیامبر ﷺ بدانها اشاره نموده بود و گفته بود: به من کمک کن، مرا از دست این دو نفر نجات بده! به آن دو گفت: از کجا آمده اید؟ گفتند: از مغرب و برای حج آمده ایم، و تصمیم گرفته ایم در جوار پیامبر ﷺ سکنی گزینیم. سلطان گفت: به من راست بگویید. از مردم پرسید: منزل آن دو کجاست؟ به وی خبر دادند که آن دو در نزدیک حجره شریف پیامبر ﷺ ساکن هستند. و اهل مدینه بخاطر نماز و روزه و صدقه بسیار آنها و نیز زیارت بقیع و قبا به مدح و ثنایشان پرداختند. سلطان آن دو را بازداشت کرد و به منزلشان رفت و به تنهایی به وارسی و جستجوی خانه مشغول شد. او حصیری را در خانه برداشت و به یکباره چشمش به تونلی افتاد که آن دو کنده بودند که به حجره شریف پیامبر ﷺ می رسید. مردم از این امر مات و مبهوت ماندند و به لرزه افتادند. سلطان در این هنگام گفت: به من راستش را بگویید! و آن دو را به شدت زد تا اینکه اعتراف کردند که هر دو مسیحی هستند، و مسیحیان آنها را همراه حاجیان مغربی به مدینه فرستاده و اموال هنگفتی بدانان داده و به آنها دستور داده اند که با حیله و نیرنگ جسد پیامبر ﷺ را سرقت کنند. آن دو شبانه مشغول کندن بوده اند و هر یک محفظه ای پوستین به سبک مغربی داشته اند که بوسیله آن خاک را از تونل بیرون می بردند و بعد با ادعای زیارت بقیع خاک را در آنجا بین قبرها خالی می کردند و مدتی مشغول این کار بوده اند. اما زمانی که به حجره شریف پیامبر ﷺ نزدیک شدند، رعد و برق آسمان شروع شد و صاعقه ای بزرگ روی داد⚡️⚡️⚡️ که مردم پنداشتند کوهها را از ریشه برکنده است. و سلطان صبح همان شب به آنجا رفت. زمانی که آن دو اعتراف کردند و حقیقتشان به دست سلطان آشکار شد و عظمت الله متعال در این مسئله را مشاهده نمود، به شدت به گریه افتاد😭 و سپس دستور داد که گردن آن دو مسیحی را بزنند.🗡 بعد فرمان داد تا سرب بسیاری را آنجا حاضر کنند و خندق بزرگی پیرامون حجره شریف پیامبر ﷺ کندند و آن سرب ها را ذوب نمودند و خندق را با آن پر کردند، و بدین ترتیب دیواری سربی دور حجره شریف قرار دادند. ایشان پس از آن به دیارشان برگشتند و دستور به تضعیف مسیحیان داد و همچنین فرمان داد که هیچ کافری در کارهای مملکت بکار گرفته نشود. 📕منبع: جمال الدین عبدالرحیم بن حسن بن علی اسنوی (ت ۷۷۲ ه) در رساله خود با عنوان «نصیحة أولی الالباب فی منع استخدام النصاری» که برخی آن را «الانتصارات الاسلامیة» می نامند، این داستان را خاطر نشان ساخته است. و علی بن عبدالله سمهودی (ت ۹۱۱ ه) در کتابش «وفاء الوفا بأخبار دار المصطفی» (۶۵۰-۶۴۸/ ۲) آن را نقل نموده است. و نیز حافظ جمال الدین عبدالله بن محمد بن احمد مطری (ت ۷۶۵ ه) که رئیس مؤذنان حرم نبوی و همچنین یک مورخ و مولف کتاب «الإعلام فیمن دخل المدینة من الأعلام» می باشد، آن را نقل نموده است و گفته: “آن را از فقیه علم الدین یعقوب بن أبوبکر از بزرگانی که حادثه را دریافته اند شنیده ام که…” و قصه را آنگونه که گذشت، بیان کرده است. #تاریخ_اسلام ',
    '♦️خطر فرونشست ۸ فرودگاه‌ کشور را تهدید می‌کند/خطر فرونشست در کمین ۵۶ هزار کیلومترمربع در کشور 🔹هیولایی به نام فرونشست در حال بلعیدن ایران است. استفاده بی‌رویه از آب‌های زیرزمینی و حفر چاه‌های مجاز و غیرمجاز جان سفره‌های زیرزمینی را گرفته و خشکی و فرونشست را به جایش هدیه داده است. 🔹براساس گزارشی رسمی سازمان حفاظت محیط‌زیست: «در حال حاضر ۲۵۶ شهر از ۴۲۹ شهر ایران درگیر مسئله فرونشست هستند 🔹در این میان، هشت شهرستان شامل کرمان، رفسنجان، و سیرجان در استان کرمان، مرودشت در استان فارس، نیشابور در استان خراسان رضوی، بوئین‌زهرا در استان قزوین و گنبدکاووس و آق‌‌قلا در استان گلستان فرونشستی به‌وسعت بیش از یک‌هزار کیلومترمربع را تجربه می‌کنند. 🔹خطوط مترو، بزرگراه‌ها، جاده‌های اصلی، خطوط راه‌آهن و جاده‌های کامیون‌رو از جمله زیرساخت‌هایی هستند که در معرض آسیب از فرونشست قرار دارند. از ۹ هزار و ۵۰۰ کیلومتر خطوط ریلی در سراسر کشور حدود ۱۵ درصد (معادل یک‌هزار و ۳۸۰ کیلومتر) در نواحی درگیر فرونشست کشیده شده‌اند. ',
    'از آقاى [ميرحسين] موسوى خواستم كه لوايح براى ارجاع داورى را كه امروز يكى از آنها تمام وقت مجلس را گرفت، به مجلس نياورند؛ چون شوراى نگهبان̛گفته، در مورد قراردادهاى̛رژيم گذشته كه در متن قرارداد، ارجاع به داورى̛پيش بينى شده و ناچاريم بپذيريم، تصويب مجلس ضرورتى ندارد. ✍آیت الله هاشمی در آینه خاطرات 🗓سه‏شنبه ۱۵ دی ۱۳۶۰ 💠به خاطر كسالت، با اينكه جلسه علنى داشتيم، در منزل ماندم. آقاى [موسوى] خوئيني‏ها وآقاى [محمد] يزدى، مجلس را صبح و عصر اداره كردند. عصر را براى جواب وزرا به سئوالات̛نمايندگان، اختصاص داده بوديم. به‌ مطالعه پرداختم و مخصوصاً مطالبى درباره امنيت خليج فارس و وحشت غربي‏ها از انقلاب اسلامى در منطقه. تلفنى با مجلس و نخست‏وزيرى براى̛كارها تماس̛داشتم. از آقاى [ميرحسين] موسوى خواستم كه لوايح براى ارجاع داورى را كه امروز يكى از آنها تمام وقت مجلس را گرفت، به مجلس نياورند؛ چون شوراى نگهبان̛گفته، در مورد قراردادهاى̛رژيم گذشته كه در متن قرارداد، ارجاع به داورى̛پيش بينى شده و ناچاريم بپذيريم، تصويب مجلس ضرورتى ندارد. محسن براى‌ ملاقات عمه‌اش به قم رفته بود. عصر همه بچه‌ها، جمع‌ شدند. عصر از دفتر مشاورت امام، سرگرد ترابى آمد و گزارشى از ارتش داد. مقدارى در حياط̛منزل، قدم زدم و براى سخنرانى روز جمعه در جلسه سفرا و خطبه‌هاى نمازجمعه اين هفته، فكر كردم. خبر مهمى در داخل نداشتيم. توسعه تظاهرات̛مسلمانان سودان، امروز مسأله مهم̛است . ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'براى تشييع جنازه مرحوم [غلامرضا] رحيمى نماينده ماهشهر به سالن رفتم. اطلاع دادند كه امام با دستور دفن در صحن [حضرت معصومه(س)] قم بدون پرداخت پول قبر موافقت نكردند. به برادرش گفتم خود ما مى‌پردازيم، در صحن دفن كنند. /آقازاده [وزير نفت] آمد. گزارشى از وضع گاز داد و درباره موافقت با كشيدن لوله گاز صادراتى به تركيه مذاكره كرديم. از صدور نفت و قيمت‌هاى نفت اظهار رضايت كرد و گفت امروز يك كشتى حامل نفت سفيد، موشك خورده است ✍آیت الله هاشمی در آینه خاطرات 🗓یکشنبه ۱۵ دی ۱۳۶۴ 💠بعد از نماز آقاى محسن رضايى آمد. در مورد عمليات آينده و همكارى با نيروهاى هوايى و دريايى و احتمال ضرورت ورود به جزيره بوبيان كويت - در صورت اشغال آن توسط عراق و براى بيرون راندن عراق - مطالبى گفت. 💠دستور جلسه مجلس، معرفى دو وزير اقتصاد و دارايى و معادن و فلزات بود كه مخالفت جدى نشد و رأى آوردند و يكسره كار كرديم. آقاى مهدوى نماينده بابلسر آمد و راجع به دانشگاه آزاد اسلامى بروجرد و حزب [جمهورى اسلامى] بروجرد و وضع روحانيت بابل و نماينده بابل صحبت كرد. 💠دو نفر از كاركنان دفترم آمدند و مدعى تبعيض بودند. عصر [آقاى تورگوت اوزال] نخست‌وزير تركيه و هيأت همراه آمدند و درباره توسعه روابط و جنگ مذاكره كرديم. ايشان مى‌گفت اگر عراق به شما خسارت جنگ را بپردازد، منجر به سقوط صدام هم مى‏شود. پرسيد آيا شرط اين است كه خود صدام برود يا حزب بعث. گفتم ممكن است به رفتن صدام قناعت شود. اين را گامى به جلو تلقى كردند. 💠براى تشييع جنازه مرحوم [غلامرضا] رحيمى نماينده ماهشهر به سالن رفتم. اطلاع دادند كه امام با دستور دفن در صحن [حضرت معصومه(س)] قم بدون پرداخت پول قبر موافقت نكردند. به برادرش گفتم خود ما مى‌پردازيم، در صحن دفن كنند. 💠آقازاده [وزير نفت] آمد. گزارشى از وضع گاز داد و درباره موافقت با كشيدن لوله گاز صادراتى به تركيه مذاكره كرديم. از صدور نفت و قيمت‌هاى نفت اظهار رضايت كرد و گفت امروز يك كشتى حامل نفت سفيد، موشك خورده است. آقاى دكتر [حسن] روحانى آمد و درباره عمليات آينده، پدافند و مسایل جنبى صحبت كرديم. 💠بعد از نماز مغرب، آقاى انوارى امام جماعت بندرعباس آمد و براى امضاى مصوبه خبرگان درباره نيابت رهبرى آيت‌الله منتظرى و در مورد تعويض امام جمعه و تعيين استاندار [هرمزگان] مذاكره كرد. تا ساعت هشت‏ونيم در دفترم كار كردم و سپس به خانه آمدم. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'سفير شوروى به ملاقات آمد. خبر از توطئه آمريكا و تجمع̛نيروهاى ضدانقلاب‌ در خاك تركيه و نفوذ دسته‏هايى از آنها به داخل مى‌داد و مى‌گفت مشابه آن در بلوچستان‌ پاكستان پايه‏ريزى شده. گفتم اين خبر از منابع̛مختلف مى‌رسد، ولى كسى سرنخ مشخصى̛نمى‌دهد. من از عدم انجام تعهد شوروى در خصوص تحويل روزانه چهل هزار بشكه‌ فراوده‌هاى نفتى به ايران،گله كردم. قرار شده به خاطر تعطيلى پالايشگاه آبادان و كرمانشاه، آنها به ما فرآورده بدهند و در مقابل از ما نفت خام بگيرند. ✍آیت الله هاشمی در آینه خاطرات 🗓دوشنبه ۱۴ دی ۱۳۶۰ 💠پس از نماز، مشغول مطالعه شدم. صبح آقاى بهزاد نبوى آمد. درباره لايحه̛مجازات اخلال̛كنندگان در كوپن‏هاى سهميه بندى كالاها، مشورت داشت. در مجلس كميسيون قضایى اين̛لايحه را رد كرده و ستاد بسيج اقتصادى، نگران̛است، اگر در جلسه علنى رد شود، اصل سهميه‌بندى مخدوش و ضعيف مى‌شود. گفتم احتياج به اصلاحات دارد. آقاى نخست‏وزير هم در همين خصوص، تلفنى صحبت كرد. در كميسيون تدوين آیين‏نامه شركت كردم. 💠بعدازظهر، سفير شوروى به ملاقات آمد. خبر از توطئه آمريكا و تجمع̛نيروهاى ضدانقلاب‌ در خاك تركيه و نفوذ دسته‏هايى از آنها به داخل مى‌داد و مى‌گفت مشابه آن در بلوچستان‌ پاكستان پايه‏ريزى شده. گفتم اين خبر از منابع̛مختلف مى‌رسد، ولى كسى سرنخ مشخصى̛نمى‌دهد. من از عدم انجام تعهد شوروى در خصوص تحويل روزانه چهل هزار بشكه‌ فراوده‌هاى نفتى به ايران،گله كردم. قرار شده به خاطر تعطيلى پالايشگاه آبادان و كرمانشاه، آنها به ما فرآورده بدهند و در مقابل از ما نفت خام بگيرند. 💠گروهى از تربيت بدنى آمدند و از احتمال عزل آقاى داودى سرپرست سازمان̛نگران‌ بودند. آقاى آخوندى از مركز جهاد تلفن كرد و از تغيير نماينده امام ناراضى بود. مصاحبه‌اى با تلويزيون درباره هفته وحدت داشتم . 💠سرهنگ كاكاوند فرمانده پايگاه هوایى بوشهر آمد و مطالبى در خصوص̛ضعف عملكرد نيروى دريايى و عدم استفاده از امكانات آنها داشت و پيشنهاداتى ارایه كرد. شب به خانه آمدم̛و تا آخر شب، اخبار گوش دادم. در حال̛نماز عشا، در مجلس دچار تب و لرز خفيفى شدم. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'ملاقاتى با سفير سوئد داشتم كه مى‏خواست موافقت ما را براى آمدن آقاى اولاف پالمه نخست‏وزير سوئد براى ميانجيگرى در جنگ جلب كنند و نظرات ما را بفهمد و تلاش داشت كه اجازه آمدن سفير انگليس به ايران را بگيرد. ✍آیت الله هاشمی در آینه خاطرات 🗓سه‏شنبه ۱۴ دی ۱۳۶۱ 💠به جلسه علنى رفتم. در ماده نه طرح مالك و مستأجر، اصلاحات مهمى شد كه به دادگاه‌ها حق مى‏دهد در مورد تشخيص عسر و حرج براى مستأجر از [صدور] حكم تخليه خوددارى نمايند. در اصل طرح سقف يك سال بود، اين سقف را برداشتيم. ظهر گروهى از نمايندگان كه به كردستان رفته بودند، راجع به اختلافات استاندار و سپاه و آقاى موسوى نماينده امام گزارشى دادند. آقاى فاضل هرندى هم از آلمان آمده و از اخلال نماينده يكى از نهادها در روابط حزب جمهورى [اسلامى] و دانشجويان مى‌گفتند. 💠عصر ملاقاتى با سفير سوئد داشتم كه مى‏خواست موافقت ما را براى آمدن آقاى اولاف پالمه نخست‏وزير سوئد براى ميانجيگرى در جنگ جلب كنند و نظرات ما را بفهمد و تلاش داشت كه اجازه آمدن سفير انگليس به ايران را بگيرد. 💠سپس با ائمه جمعه گيلان ملاقات داشتم كه از حزب [جمهورى اسلامى] شكايت داشتند و استاندار. ملاقاتى با ائمه جمعه اهل سنت سيستان داشتم كه خواهان توجه بيشتر به اهل سنت سيستان بودند و اعلام وفادارى مى‌نمودند. 💠بعد از نماز مغرب با سفير آلمان شرقى ملاقات كردم كه از طرف ریيس مجلس آلمان شرقى مرا به آلمان دعوت كرد و شرحى از كمك‏شان در جنگ داد. 💠چند نفر از جمعيت مبارزه با [استعمال] دخانيات آمدند و براى شروع [كار] مشورت كردند. كمك خواستند. آمارى دادند. قرار شد با امام صحبت كنم. در مجلس هم نهضتى [براى مبارزه با استعمال دخانيات] به‏وجود آمده. شب به منزل آمدم. از جهاد سازندگى كهنوج آمدند و گزارشى از كارهاى عمرانى كهنوج دادند كه جالب بود. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'درباره داورى لاهه جلسه داشتيم؛ تصميم گرفتيم از الجزاير بخواهيم قبل از پرداخت به اشكالات احكام توجه كند و به طور اتوماتيك احكام را نپردازد. ✍آیت الله هاشمی در آینه خاطرات 🗓چهار‌‏شنبه ۱۴ دی ۱۳۶۲ 💠تا ظهر در خانه بودم. گزارش‏هاى واصله را خواندم. تبليغات استكبارى دنيا، جمهورى اسلامى را به عنوان تروريست و ماجراجو و جنگ‌طلب، هدف [حمله] قرار داده است. هم از ما و اسلام مى‌ترسند و هم قدرت رويارويى مستقيم را ندارند، بنابراين از اين راه‌ها وارد شده‌اند. نگران پيروزى ما در جنگ با عراق هستند. مقدارى به كتابخانه‌ام پرداختم. سعيد [لاهوتى] به عنوان مرخصى از سربازى آمده و با فاطى به منزل خودشان رفتند. عفت هم به عنوان پياده‏روى به آنجا رفت. 💠پيش از ظهر آقاى [عبدالمجيد] معاديخواه آمد و طرح كار بنياد تدوين تاريخ را ارایه و مشورت كرد. ناهار را با احمدآقا و شوراى عالى قضايى مهمان آقاى توسلى بوديم. ساعت سه بعداز ظهر آقاى طهماسبى از اصفهان آمد و اشكالات در راه پياده كردن اختراع نيمچه تانك را داد؛ بهانه‌گير، ولى كارش مهم است، بايد با او ساخت. طلبه‌اى است با روحيه ابتكارى. 💠عصر در جلسه هيأت ریيسه شركت كردم و شب در دفترم ماندم. گزارش‏ها را خواندم. امروز اسرایيل با شانزده هواپيما به مواضع نيروهاى اسلامى در بعلبك حمله كرده است. گزارش‏ها تعداد شهداى مردم بعلبك را زياد گفته‌اند؛ شام مهمان نخست‏وزير بوديم. قبل از شام جلسه هيأت امناى دانشگاه آزاد اسلامى برگزار شد. درباره بودجه و جلسات تك‏درسى و گزينش، مصوباتى داشتيم. سپس با حضور دكتر [گودرز] افتخار[جهرمى] و دكتر [محمود] كاشانى و دكتر [حميد] بهرامى و سران سه قوه درباره داورى لاهه جلسه داشتيم؛ تصميم گرفتيم از الجزاير بخواهيم قبل از پرداخت به اشكالات احكام توجه كند و به طور اتوماتيك احكام را نپردازد. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی ',
    'احمدآقا و من به آقاى موسوى اردبيلى در مورد اشتباه بودن بازداشت شهردار خوب اصفهان گفتيم. بعد از جلسه با احمدآقا درباره سيد هادى هاشمى صحبت كرديم. نظر هر دوى ما اين است كه بهتر است ايشان آزاد شود. ✍آیت الله هاشمی در آینه خاطرات 🗓یکشنبه ۱۴ دی ۱۳۶۵ 💠مهدى بعد از دو سه هفته، امروز به مدرسه رفت. ساعت هفت‏ونيم به مجلس رسيدم. قبل از دستور، رحلت آقاى [آيت‌الله حاج شيخ عبدالرحمن] حيدرى امام جمعه ايلام [و نماينده مجلس خبرگان] را تسليت گفتم. تذكرات نمايندگان را كه تبريك و تسليت شهداى كربلاى چهار بود، نخواندم. چون نشان كثرت شهدا است و مطابق آیين‌نامه، اجبارى به خواندن نيست. قانون محاسبات در دستور بود. به خاطر اهميت ماده 30 كه مربوط به ذيحسابى است، تا آخر در جلسه ماندم و تعديل شد. ظهر آقاى حميدزاده آمد. از تربيت بدنى و سازمان محيط زيست گله داشت. عصر آقاى امراللهى آمد. از ندادن بودجه به انرژى اتمى گله داشت. آقاى [جليل] بشارتى مسئول پيگيرى نخست‏وزيرى آمد. درباره خسارات سيل و كمك‏هاى مردمى و برنامه امداد و بازسازى توضيح داد. مدعى است خسارات از دوازده ميليارد تومان مى‌گذرد. 💠شب سران قوا مهمان من در مجلس بودند. شرح طرح عمليات آينده و كربلاى چهار را دادم. در اين جلسه اساسنامه دانشگاه آزاد اسلامى تصويب شد. درباره مسایل مختلف بحث كرديم. احمدآقا و من به آقاى موسوى اردبيلى در مورد اشتباه بودن بازداشت شهردار خوب اصفهان گفتيم. بعد از جلسه با احمدآقا درباره سيد هادى هاشمى صحبت كرديم. نظر هر دوى ما اين است كه بهتر است ايشان آزاد شود. 💠شب در مجلس ماندم و كارهاى عقب‏مانده را انجام دادم. عفت هنوز در رفسنجان است. تلفنى با اخوى محمد، درباره نامه مسعود [همشيره‌زاده] كه از آلمان نوشته است، صحبت كرديم. ️🏻«بر ایران چه گذشت» به روایت خاطرات روزانه آیت‌ الله هاشمی رفسنجانی @khaterat_Hashemi_rafsanjani #خانه‌موزه‌آیت‌الله‌هاشمی‌رفسنجانی '
]

In [ ]:
without_emojis = []
for news in fake_news_persian:
    without_emojis.append(remove_emojis(news))
    print(remove_emojis(news))

 تلخ از تو شیرین میشود کفر از تو چون دین میشود خار از تو نسرین میشود چیزی بده درویش را جانِ من و جانان من کفر من و ایمان من سلطانِ سلطانان من چیزی بده درویش را #مولانا با آرزوی آدینه ای سرشار ازآرامش؛ از شما دعوت می شود که تاریخ امروز ایران و جهان را مرور نمائید #خورشیدی: آدینه، ۱۴ دی ۱۴۰۳/۱۰/۱۴ #میلادی: Friday, January 3, 2025-01-03 قمری: الجمعة، ٢ رجب ١٤٤٦/۰۷/۰۲ #روزها روز جشن سیرسور جشن گياه‌خواری ايرانيان #رویدادها‍ ۱۲۸۶ خ- روزنامه صور اسرافیل با همکاری میرزا جهانگیرخان شیرازی و قاسم‌خان تبریزی با همکاری میرزا علی اکبر خان قزوینی (دهخدا) صور اسرافیل را منتشر کردند ۱۲۹۲ خ- پرواز اولین هواپیما در آسمان ایران توسط کوزمینسکی، خلبان لهستانی‌الاصل   در پی اجرای يك برنامه پرواز نمايشی در آسمان تهران توسط خلبانی لهستانی با يك هواپيمای دوباله سبك در زمان احمدشاه ۱۳۰۰ خ- تشکیل ارتش شاهنشاهی ایران ۱۳۵۷ خ-  آغاز کنفرانس گوادلوپ دربارهٔ انقلاب ۱۳۵۷ ایران ۱۵۲۱ م- پاپ لئوی دهم، مارتین لوتر را تکفیر و از کلیسای کاتولیک رم طرد کرد ۱۹۹۰ م- ژنرال مانوئل نوریگا پس از مداخله نظامی آمریکا از رهبری پانام

In [ ]:
# Initialize Hazm objects
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stopwords = stopwords_list()

# Add additional stopwords
extra_stopwords = ['[', ']', '«', '»', '،', '؛', '.', ',', '؟', '!', '(', ')', 'ـ', '٪', '/', ':']
stopwords.extend(extra_stopwords)

# Read the stopwords from the file
with open('/content/persianST.txt', 'r', encoding='utf-8') as file:
    file_stopwords = file.read().splitlines()  # Read each line as a separate word

# Convert both the stopwords and file stopwords to sets
stopwords_set = set(stopwords)
file_stopwords_set = set(file_stopwords)

# Find the intersection (common words)
stopwords = stopwords_set.intersection(file_stopwords_set)

# Function to normalize, tokenize, lemmatize, remove stopwords, and clean unwanted patterns
def process_text(text):
    """
    Process the input text: normalize, remove unwanted patterns, tokenize, lemmatize, and remove stopwords.

    Args:
        text (str): The input Persian text.

    Returns:
        dict: A dictionary with normalized text, tokens, filtered tokens, lemmatized tokens, and cleaned text.
    """
    # Step 1: Normalize the text
    normalized_text = normalizer.normalize(text)

    # Step 2: Remove zero-width characters (\u200c, \u200b, \u200d) and unwanted patterns
    cleaned_normalized_text = re.sub(r'[\u200c\u200b\u200d]|(?:\b[مخ]-)', '', normalized_text)

    # Step 3: Tokenize the text (split into words)
    tokens = word_tokenize(cleaned_normalized_text)

    # Step 4: Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stopwords]

    # Step 5: Lemmatization (reduce words to base form)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Step 6: Reconstruct the cleaned text (using lemmatized tokens)
    cleaned_text = ' '.join(lemmatized_tokens)

    # Return the results in a dictionary
    return {
        'normalized_text': normalized_text,
        'cleaned_normalized_text': cleaned_normalized_text,  # Text after removing zero-width characters and patterns
        'tokens': tokens,
        'filtered_tokens': filtered_tokens,  # Tokens after stopword removal
        'lemmatized_tokens': lemmatized_tokens,  # Tokens after lemmatization
        'cleaned_text': cleaned_text  # Final cleaned text after processing
    }

In [ ]:
output = []
for news in without_emojis:
  output.append(process_text(news)['cleaned_text'])
pd.DataFrame(output)

,0
0,تلخ تو شیرین میشود کفر تو دین میشود خار تو نسر...
1,تلخ تو شیرین میشود کفر تو دین میشود خار تو نسر...
2,گروه مدلل ( مالک برد#بر غنچه ) شرکت صافولا بهش...
3,توضیحات کامل شهردار رباطکریم خصوص ۲ پروژه شاخص...
4,آقای [ جلیل ] بشارت ، [ رییس دفتر مناطق محروم ...
5,موسوی لاری ، وزیر کشور معاون تاجزاده ، گزارش پ...
6,اظهار امیدواری عرفات توافق اسراییل / هشدارسران...
7,اخبار شب آمریکا جنگنده لیبی مدیترانه سرنگون کر...
8,محسن . سیمان شکر صحبت کرد#کن . میخواست [ آموزش...
9,دکتر ملکی برادر آمد#آ ؛ پزشک هستنند کانادا کار...


In [ ]:
def compute_persian_word_frequency(text):
    # Convert text to lowercase and remove non-alphabetic characters (keeping spaces)
    text = re.sub(r'[^آ-ی\s]', '', text.lower())

    # Split the text into words
    words = text.split()

    # Count the frequency of each word
    word_freq = Counter(words)

    # Convert the word frequency to a DataFrame
    word_freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])

    return word_freq_df

In [ ]:
compute_persian_word_frequency(output[0]).sort_values(by = 'Frequency', ascending = False)

,Word,Frequency
22,ایران,6
3,میشود,4
111,ایرانی,4
1,تو,3
132,خواننده,3
...,...,...
85,لوتر,1
86,تکفیر,1
87,کلیسا,1
88,کاتولیک,1
